# Data Quality Analysis - Bronze Layer

This notebook performs comprehensive data quality checks on the raw data files to identify:
- Missing values
- Data Type Validation
- Non-numeric values in numeric columns
- Duplicate records

## 1. Import Required Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Data Files

In [2]:
# Define data directory
data_dir = "data"

# Load all data files
df_attributes = pd.read_csv(os.path.join(data_dir, "features_attributes.csv"))
df_financials = pd.read_csv(os.path.join(data_dir, "features_financials.csv"))
df_clickstream = pd.read_csv(os.path.join(data_dir, "feature_clickstream.csv"))
df_loan_daily = pd.read_csv(os.path.join(data_dir, "lms_loan_daily.csv"))

# Display dataset shapes
print("Dataset Shapes:")
print(f"  Attributes: {df_attributes.shape}")
print(f"  Financials: {df_financials.shape}")
print(f"  Clickstream: {df_clickstream.shape}")
print(f"  Loan Daily: {df_loan_daily.shape}")

Dataset Shapes:
  Attributes: (12500, 6)
  Financials: (12500, 22)
  Clickstream: (215376, 22)
  Loan Daily: (137500, 11)


## 3. Missing Values Analysis

In [4]:
def analyze_missing_values(df, dataset_name):
    """Analyze and display missing values in a dataframe"""
    missing = pd.DataFrame({
        'Column': df.columns,
        'Missing_Count': df.isnull().sum(),
        'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
    })
    missing = missing[missing['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
    
    print(f"\nMissing Values in {dataset_name} Dataset:")   
    if len(missing) > 0:
        print(missing.to_string(index=False))
    else:
        print("No missing values found!")
    
    return missing

# Analyze missing values for all datasets
missing_attributes = analyze_missing_values(df_attributes, "Attributes")
missing_financials = analyze_missing_values(df_financials, "Financials")
missing_clickstream = analyze_missing_values(df_clickstream, "Clickstream")
missing_loan_daily = analyze_missing_values(df_loan_daily, "Loan Daily")


Missing Values in Attributes Dataset:
No missing values found!

Missing Values in Financials Dataset:
      Column  Missing_Count  Missing_Percentage
Type_of_Loan           1426               11.41

Missing Values in Clickstream Dataset:
No missing values found!

Missing Values in Loan Daily Dataset:
No missing values found!


## 4. Data Type Validation

In [5]:
# Define expected numeric columns for each dataset
expected_numeric_columns = {
    'Attributes': ['Age'],
    'Financials': ['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
                   'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 
                   'Delay_from_due_date', 'Num_of_Delayed_Payment', 
                   'Changed_Credit_Limit', 'Num_Credit_Inquiries', 
                   'Outstanding_Debt', 'Credit_Utilization_Ratio', 
                   'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance'],
    'Loan_Daily': ['tenure', 'installment_num', 'loan_amt', 'due_amt', 
                   'paid_amt', 'overdue_amt', 'balance']
}

# Display current data types
print("\nCurrent Data Types:")

for name, df in [('Attributes', df_attributes), ('Financials', df_financials), 
                 ('Loan_Daily', df_loan_daily)]:
    print(f"\n{name}:")
    print(df.dtypes)


Current Data Types:

Attributes:
Customer_ID      object
Name             object
Age              object
SSN              object
Occupation       object
snapshot_date    object
dtype: object

Financials:
Customer_ID                  object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
snapshot_date                object
dty

## 5. Check for Non-Numeric Values in Numeric Columns

In [11]:
def check_non_numeric_values(df, columns, dataset_name):
    """Check for non-numeric values in columns that should be numeric"""
    print(f"\nChecking Non-Numeric Values in {dataset_name} Dataset:")
    
    issues_found = False
    
    for col in columns:
        if col not in df.columns:
            continue
            
        # Convert column to string and check if values can be converted to numeric
        temp_series = df[col].astype(str).str.strip()
        
        # Filter out NaN/None values
        temp_series = temp_series[temp_series != 'nan']
        
        # Try to convert to numeric
        numeric_conversion = pd.to_numeric(temp_series, errors='coerce')
        non_numeric_mask = numeric_conversion.isna() & (temp_series != '')
        
        if non_numeric_mask.sum() > 0:
            issues_found = True
            non_numeric_values = df[col][non_numeric_mask.values].unique()
            print(f"\n{col}:")
            print(f"  Count: {non_numeric_mask.sum()}")
            print(f"  Sample values: {list(non_numeric_values[:10])}")
    
    if not issues_found:
        print("\nAll numeric columns contain valid numeric values!")

# Check each dataset
check_non_numeric_values(df_attributes, expected_numeric_columns['Attributes'], "Attributes")
check_non_numeric_values(df_financials, expected_numeric_columns['Financials'], "Financials")
check_non_numeric_values(df_loan_daily, expected_numeric_columns['Loan_Daily'], "Loan Daily")


Checking Non-Numeric Values in Attributes Dataset:

Age:
  Count: 637
  Sample values: ['40_', '46_', '3843_', '30_', '24_', '22_', '32_', '26_', '34_', '41_']

Checking Non-Numeric Values in Financials Dataset:

Annual_Income:
  Count: 859
  Sample values: ['52312.68_', '49718.55_', '20652.98_', '28315.95_', '43062.54_', '7645.095_', '93847.86_', '11731.585_', '30739.3_', '28352.24_']

Num_of_Loan:
  Count: 623
  Sample values: ['1_', '6_', '2_', '4_', '3_', '5_', '7_', '0_', '9_', '8_']

Num_of_Delayed_Payment:
  Count: 374
  Sample values: ['18_', '20_', '0_', '9_', '19_', '15_', '16_', '12_', '17_', '11_']

Changed_Credit_Limit:
  Count: 254
  Sample values: ['_']

Outstanding_Debt:
  Count: 139
  Sample values: ['2699.17_', '642.42_', '755.17_', '865.3_', '149.92_', '3375.66_', '1559.16_', '30.31_', '4319.35_', '2817.77_']

Amount_invested_monthly:
  Count: 558
  Sample values: ['__10000__']

Monthly_Balance:
  Count: 1
  Sample values: ['__-333333333333333333333333333__']

Check

## 6. Check for Duplicate Records

In [9]:
# Check duplicates based on key columns
if 'Customer_ID' in df_attributes.columns:
    dup_attributes = df_attributes.duplicated(subset=['Customer_ID']).sum()
    print(f"\nAttributes - Duplicates based on Customer_ID: {dup_attributes}")

if 'Customer_ID' in df_financials.columns:
    dup_financials = df_financials.duplicated(subset=['Customer_ID']).sum()
    print(f"Financials - Duplicates based on Customer_ID: {dup_financials}")

if 'Customer_ID' in df_clickstream.columns:
    dup_clickstream = df_clickstream.duplicated(subset=['Customer_ID']).sum()
    print(f"Clickstream - Duplicates based on Customer_ID: {dup_clickstream}")

if 'loan_id' in df_loan_daily.columns and 'snapshot_date' in df_loan_daily.columns:
    dup_loan = df_loan_daily.duplicated(subset=['loan_id', 'snapshot_date']).sum()
    print(f"Loan Daily - Duplicates based on (loan_id, snapshot_date): {dup_loan}")


Attributes - Duplicates based on Customer_ID: 0
Financials - Duplicates based on Customer_ID: 0
Clickstream - Duplicates based on Customer_ID: 206402
Loan Daily - Duplicates based on (loan_id, snapshot_date): 0
